# 5th Pipeline: Pyserini BM25 + SPLADE Reranking + MonoT5 Reranking

Author: Monique Monteiro (moniquelouise@gmail.com)

Inspired by Leonardo Avila's notebook (https://colab.research.google.com/drive/1o-aMaptESHNLH9w9wUcO5Wz0N_VCChlG?usp=sharing#scrollTo=fYzK8SB9QG7l)

In [1]:
%%shell
pip install transformers -q
pip install datasets -q
pip install evaluate -q
pip install trectools -q
pip install torch -q
pip install faiss-cpu -q
pip install pyserini -q
pip install beir -q
pip install sentence-transformers -q
pip install git+https://github.com/naver/splade.git -q
pip install git+https://github.com/zetaalphavector/InPars.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [7]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import os
import json
import numpy as np
import pandas as pd
import random
import torch
from time import time
import collections
import evaluate
import datasets
import shutil
import pickle
import numba
import inpars

from collections import defaultdict, Counter
from datasets import load_dataset
from tqdm import tqdm
from operator import itemgetter

from torch import nn
from torch import optim
from transformers import BatchEncoding, get_linear_schedule_with_warmup

from pyserini.search.lucene import LuceneSearcher

In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade
from torch.utils.data import Dataset, DataLoader

In [4]:
from inpars import rerank, utils

In [5]:
#model_name ='castorini/monot5-3b-msmarco-10k'
model_name ='castorini/monot5-3b-med-msmarco'
#input_run = "/content/run.bm25.splade.txt"
#output_run = f"run.monot5.3b.msmarco.10k.txt"
fp16 = True
torch_compile = True
batch_size = 128
top_k = 30

In [94]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_t5 = rerank.Reranker.from_pretrained(
    model_name_or_path=model_name,
    batch_size=batch_size,
    fp16=fp16,
    device=device,
    torch_compile=torch_compile,
)

In [8]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-10"

In [9]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## NDCG@10 Evaluation Code

In [28]:
import jsonlines

query_ids = []
query_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)
     

In [11]:

import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [13]:

from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

## Pyserini BM25 Search Code

In [15]:
from pyserini.search.lucene import LuceneSearcher

In [16]:
def search_with_bm25(query,k = 1000, index_name='beir-v1.0.0-trec-covid.flat'):
  if index_name == 'beir-v1.0.0-trec-covid.flat':   
    searcher = LuceneSearcher.from_prebuilt_index(index_name)
  else:
    searcher = LuceneSearcher(index_name)
  hits = searcher.search(query, k)
  return hits

## SPLADE

In [82]:
#from transformers import BertTokenizer, BertForMaskedLM, DistilBertTokenizer, DistilBertForMaskedLM
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [84]:
from splade.models.transformer_rep import Splade

In [95]:
model_name_1 = 'naver/splade_v2_distil' 
model_name_2 = 'naver/splade-cocondenser-selfdistil'
model_name_3 = 'naver/splade-cocondenser-ensembledistil' 

#tokenizer_1 = DistilBertTokenizer.from_pretrained(model_name_1)
#model_1 = DistilBertForMaskedLM.from_pretrained(model_name_1)

#tokenizer_2 = BertTokenizer.from_pretrained(model_name_2)
#model_2 = BertForMaskedLM.from_pretrained(model_name_2)

tokenizer_3 = AutoTokenizer.from_pretrained(model_name_3)
#model_3 = AutoModelForMaskedLM.from_pretrained(model_name_3)
model_3 = Splade(model_name_3, agg="max").to(device)
model_3.eval()

In [97]:
model_splade = model_3
tokenizer_splade = tokenizer_3

In [31]:
max_length=256

In [126]:
from torch.nn.functional import relu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def vectorize_to_sparse(text, tokenizer=tokenizer_splade, model=model_splade, remove_special_tokens=False):
  # Kudos to Marcos Piau
  with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=True):
    tokenized_text = tokenizer(text, max_length=max_length, truncation=True, 
                              return_tensors='pt', 
                              #return_special_tokens_mask=True
                               ).to(device)
    
    model.to(device)
    model.eval()
    
    with torch.no_grad():
      #output = model(tokenized_text['input_ids'].to(device), 
       #              attention_mask=tokenized_text['attention_mask'].to(device))
       output = model(q_kwargs=tokenized_text)["q_rep"]
       print("output = ", output)
       print("type(output) = ", type(output))
       #doc_rep = model(d_kwargs=tokenizer(doc, return_tensors="pt"))["d_rep"].squeeze()


    #w_ij = output.logits[0,:]

    #mask_tokens_validos = 1 - tokenized_text['special_tokens_mask'].to(device)

    #Kudos Leandro Carísio
    #1) Removes all dimenstions of size 1 (here, it's the first one)
    #2) Adds a 1 dimension to the end (equivalent to transpose, I think)
    #3) Replicates (expands) up to the vocabulary size -> 5 X 30k 
    #if remove_special_tokens:
     # mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
    #else:
     # mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
      #mask = torch.ones(mask.size()).to(device)

    #wj = torch.max(torch.log(1 + relu(w_ij*mask)), dim=0)[0]

  #According to ChatGPT: In PyTorch, the to_sparse() method is used to convert a 
  #dense tensor into a sparse tensor.  A dense tensor is a tensor that contains 
  #all elements, including those with zero values. In contrast, a sparse tensor 
  #is a tensor that only stores the non-zero elements, along with their indices. 
  #Sparse tensors are useful when working with large tensors with mostly zero 
  #values, as they can save memory and computational resources.
  #return wj.to_sparse()

  return torch.transpose(output, 0, 1).squeeze().to_sparse()

In [22]:
import jsonlines

passage_ids = []
passage_texts = []
id_to_text = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_text[id] = text

#Sorts the passages by length
passage_indices = sorted(range(len(passage_texts)), 
                         key=lambda k: len(passage_texts[k]))
passage_texts = sorted(passage_texts, key=lambda k: len(k))
passage_ids = sorted(passage_ids, key=lambda k: len(id_to_text[k]))

In [21]:
def search_by_query_vector_in_inverted_index(inverted_index, query_vec, k, ids=None):
  query_vec = query_vec.coalesce()
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  doc_ids = []
  indices = query_vec.indices()[0]
  values = query_vec.values()

  for token_id, wj in zip(indices, values):
    token_id = token_id.item()
    wj = wj.item()
    
    if token_id in inverted_index:
      doc_ids = inverted_index[token_id]["docs"]
      wjs = inverted_index[token_id]["wj"]

      for idx, doc_wj in zip(doc_ids, wjs):
        if ids is not None and passage_ids[idx] in ids:
          doc_scores[passage_ids[idx]] += wj * doc_wj
        elif ids is None:
          doc_scores[passage_ids[idx]] += wj * doc_wj
        
  doc_scores = dict(sorted(doc_scores.items(), key=lambda x:x[1], 
                           reverse=True)[:k])
          
  return doc_scores

In [23]:
import array
import pandas as pd
from collections import defaultdict
from collections import Counter
import pickle
import os

def load_or_build_inverted_index(index_path = f"{main_dir}/index.pickle", 
                                 docs_matrix=None):
  if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      index = pickle.load(f)
  else:
    print("Building inverted index...")
    inverted_index = dict()
    idx = 0

    def process(doc_id, idx):
      assert passage_ids[idx] == doc_id
      doc_vec = docs_matrix[idx]
      doc_vec = doc_vec.coalesce()
      indices = doc_vec.indices()[0]
      values = doc_vec.values()

      for token_id, wj in zip(indices, values):
        token_id = token_id.item()
        wj = wj.item()
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["docs"].append(idx)
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["wj"].append(wj)

    for i in tqdm(range(docs_matrix.shape[0])):
      process(passage_ids[i], i)
      
    index = {"inverted_index": inverted_index}

    with open(index_path, "wb") as f:
      pickle.dump(index, f)

  return index

In [25]:
start = time()
index = load_or_build_inverted_index()
end = time()
print("Time spent to build inverted index = ", end - start)

Loading index...
Time spent to build inverted index =  3.060671091079712


In [26]:
inverted_index = index["inverted_index"]

## Pipeline

In [14]:
k=1000

In [133]:
import time

def search_bm25_splade_monot5(query_id, query, top_k=1000, 
                       index_name='beir-v1.0.0-trec-covid.flat'):
  #1st step: searches with BM25
  bm25_hits = search_with_bm25(query, k=top_k+1, index_name=index_name)
 
  if index_name == 'beir-v1.0.0-trec-covid.flat':
    ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
  else:
    ids = [json.loads(bm25_hits[i].raw)['id'] for i in range(len(bm25_hits))]
  ids = set(ids)

  query_embedding = vectorize_to_sparse(query, tokenizer=tokenizer_splade, model=model_splade)
  
  doc_scores = search_by_query_vector_in_inverted_index(inverted_index, query_embedding, k+1, 
                                                        ids=ids)
  
  input_run = "run_bm25_splade.csv"
  rank = 1
  with open(input_run, 'w') as f:
    for doc_id, score in doc_scores.items():
        f.write(f'{query_id} Q0 {doc_id} {rank} {score} "bm25_splade"\n')
        rank += 1
  

  run = utils.TRECRun(input_run)
  topics = {query_id: query}

  run.rerank(model_t5, topics, id_to_text, top_k=top_k)
  output_run = f"run.monot5.3b.msmarco.10k.txt"
  run.save(output_run)

  doc_scores = dict()
  with open(output_run, 'r') as f:
    for line in f:
      fields = line.strip().split()
      doc_id =fields[2]
      score = fields[4]
      doc_scores[doc_id] = score
  
  return doc_scores

In [134]:
from collections import defaultdict

run_bm25_splade_monot5 = defaultdict(list)

start = time.time()
k=1000

for i, query in tqdm(zip(query_ids, query_texts)):
  doc_scores = search_bm25_splade_monot5(i, query)
  n = len(doc_scores)
  run_bm25_splade_monot5["query"] += [i] * n
  run_bm25_splade_monot5["docid"] += doc_scores.keys()
  run_bm25_splade_monot5["score"] += doc_scores.values()
  run_bm25_splade_monot5["q0"] += ["q0"] * n
  run_bm25_splade_monot5["rank"] += list(range(1,n+1))
  run_bm25_splade_monot5["system"] += ["doc2query_bm25_splade_inpars"] * n

end = time.time()
print("Time spent = ", end - start)
print("Time spent by query = ", (end - start)/len(query_ids))

0it [00:00, ?it/s]

output =  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
type(output) =  <class 'torch.Tensor'>


Rescoring:   0%|          | 0/8 [00:00<?, ?it/s]

1it [00:20, 20.10s/it]

output =  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
type(output) =  <class 'torch.Tensor'>


...

49it [17:13, 20.97s/it]

output =  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
type(output) =  <class 'torch.Tensor'>


Rescoring:   0%|          | 0/8 [00:00<?, ?it/s]

50it [17:35, 21.10s/it]

Time spent =  1055.059954881668
Time spent by query =  21.101199097633362


In [135]:
eval_ndcg10(run_bm25_splade_monot5)

0.784231574172726

In [136]:
!nvidia-smi

Wed May 10 20:42:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    54W / 400W |  35475MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------